In [1]:
from serpapi import GoogleSearch
import json
import os
from dotenv import load_dotenv
from anthropic import Anthropic


load_dotenv()

True

In [2]:

CLAUDE_API_KEY = os.environ.get("CLAUDE_API_KEY")
client = Anthropic(api_key=CLAUDE_API_KEY)

system_prompt = ""

try:
    with open("hotels_request_gen_prompt.txt", "r", encoding="utf-8") as f:
        system_prompt = f.read()
        
except FileNotFoundError:
    print("hotels_request_gen_prompt.txt not found")

else:
    def get_hotel_request(user_input):
        response = client.messages.create(
            model="claude-sonnet-4-20250514",   
            max_tokens=500,
            system=system_prompt,
            messages=[
                {"role": "user", "content": user_input}
            ]
        )
        raw = response.content[0].text.strip()
        return raw



In [3]:
import ast


user_text = "Book me a hotel in Dubai for 2 adults from Jan 10/2026 to Jan 15/2026, in price range of 250 to 400 SAR per night, with free cancellation and and pool and Free Wi-Fi."
params = get_hotel_request(user_text)

out_file = "hotel_params.json"

# convert params string to dict
if isinstance(params, str):
    s = params.strip()
    if s.startswith("params"):
        s = s.split("=", 1)[1].strip()
    try:
        params_dict = ast.literal_eval(s)
    except Exception:
        params_dict = json.loads(s)
else:
    params_dict = params

# write dict to json file
with open(out_file, "w", encoding="utf-8") as f:
    json.dump(params_dict, f, ensure_ascii=False, indent=2)
print("Hotel parameters written to hotel_params.json")


Hotel parameters written to hotel_params.json


In [4]:
import ast
SERPAPI_KEY = os.environ.get("SERPAPI_KEY")

# If get_hotel_request returned the dict as a string like "params = {...}", convert it to a dict
if isinstance(params, str):
    s = params.strip()
    if s.startswith("params"):
        s = s.split("=", 1)[1].strip()
    params = ast.literal_eval(s)

print("Search Parameters:", params)
params["api_key"] = SERPAPI_KEY


Search Parameters: {'engine': 'google_hotels', 'q': 'Hotels in Dubai', 'check_in_date': '2026-01-10', 'check_out_date': '2026-01-15', 'adults': '2', 'children': '0', 'min_price': '250', 'max_price': '400', 'free_cancellation': 'true', 'amenities': ['6', '35'], 'currency': 'SAR', 'gl': 'sa', 'hl': 'en', 'api_key': ''}


In [5]:
search = GoogleSearch(params)
results = search.get_dict()

with open("hotels.json", "w") as f:
    json.dump(results, f, indent=2)

In [6]:
clean_hotels = []
for hotel in results.get("properties", []):
    clean_hotels.append({
        "id": hotel.get("property_token"),
        "name": hotel.get("name"),
        "price": hotel.get("rate_per_night", {}).get("extracted_lowest"),
        "stars": hotel.get("extracted_hotel_class"),
        "rating": hotel.get("overall_rating"),
        "reviews": hotel.get("reviews"),
        "location_rating": hotel.get("location_rating"),
        "coordinates": hotel.get("gps_coordinates"),
        "amenities": hotel.get("amenities", []),
        "link": hotel.get("link"),
        "sponsored": hotel.get("sponsored")
    })


out_file = "hotels_filtered.json"
with open(out_file, "w", encoding="utf-8") as f:
    json.dump(clean_hotels, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(clean_hotels)} hotels to {out_file}")


Wrote 18 hotels to hotels_filtered.json


In [7]:
import os, json, ast, re
from anthropic import Anthropic

CLAUDE_API_KEY = os.environ.get("CLAUDE_API_KEY")
client = Anthropic(api_key=CLAUDE_API_KEY)

system_prompt = ""

try:
    with open("top_hotels_prompt.txt", "r", encoding="utf-8") as f:
        system_prompt = f.read()

except FileNotFoundError:
    print("top_hotels_prompt.txt not found")

else:
    def get_best_hotels(hotels, preferences, top_n=5):
        user_input = (
            f"Here is the list of hotels: {json.dumps(hotels, ensure_ascii=False)}\n"
            f"User preferences: {preferences}\n"
            f"Please select the top {top_n} hotels that best match these preferences "
            f"and return only a Python dictionary in the specified format."
        )

        response = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=500,
            system=system_prompt,
            messages=[
                {"role": "user", "content": user_input}
            ]
        )

        raw = response.content[0].text.strip()

        # --- clean output ---
        raw = raw.strip("` \n")  # remove markdown fences
        if isinstance(raw, str) and "=" in raw.splitlines()[0]:
            raw = raw.split("=", 1)[1].strip()

        # Try direct parse
        parsed = None
        for parser in (ast.literal_eval, json.loads):
            try:
                parsed = parser(raw)
                break
            except Exception:
                pass

        # Extract dictionary or list if surrounded by text
        if not parsed:
            m = re.search(r'(\{.*\}|\[.*\])', raw, re.DOTALL)
            if m:
                content = m.group(1)
                for parser in (ast.literal_eval, json.loads):
                    try:
                        parsed = parser(content)
                        break
                    except Exception:
                        pass

        # --- final normalization ---
        if isinstance(parsed, dict) and "top_hotels" in parsed:
            return parsed["top_hotels"]
        elif isinstance(parsed, list):
            return parsed
        else:
            print("Failed to parse LLM output.")
            print(raw)  # print full raw output to debug
            return []


In [8]:
best_hotels = get_best_hotels(clean_hotels, user_text)
best_hotels

[{'id': 'ChkI5YjzyMic_9R2Gg0vZy8xMWYycF9obTEyEAE',
  'name': 'Queen Elizabeth 2',
  'reason': 'Best value at 293 SAR with excellent 4.4 rating, indoor pool, and Free Wi-Fi - historic floating hotel experience'},
 {'id': 'ChkIzq2k_vTpiYPdARoML2cvMWhjMjFqNHdjEAE',
  'name': 'Mercure Gold Hotel Al Mina Road Dubai',
  'reason': 'Great value at 315 SAR with 4.3 rating, excellent 4.3 location rating, outdoor pool and Free Wi-Fi'},
 {'id': 'ChgI2Lbw6vWht6eVARoLL2cvMXZibm13cHYQAQ',
  'name': 'Howard Johnson by Wyndham Bur Dubai',
  'reason': 'Excellent value at 319 SAR with high 4.4 rating, pools, and Free Wi-Fi - great overall package'},
 {'id': 'ChoI1NyP9aT5nZ_TARoNL2cvMTFnZjlmbnIwbBAB',
  'name': 'Rove At The Park',
  'reason': 'Outstanding 4.8 rating at 351 SAR with outdoor pool, Free Wi-Fi, and modern amenities'},
 {'id': 'ChkIrd6Qg8faheJPGg0vZy8xMWZzYjF6Y2J5EAE',
  'name': 'Holiday Inn Dubai al-Maktoum Airport by IHG',
  'reason': 'High 4.7 rating at 343 SAR with outdoor pool, Free Wi-Fi